<a href="https://colab.research.google.com/github/deepakri201/SR_for_Lung_PET_CT_Dx/blob/main/SR_for_Lung_PET_CT_Dx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DICOM SR lesion bounding boxes for Lung-PET-CT-Dx collection

This notebook creates bounding box SR annotations from the Lung-PET-CT-Dx collection. TCIA provides xml files containing the annotations in an xml file per SOPInstanceUID.

Code for parsing the XML files comes from [TCIA website](https://www.cancerimagingarchive.net/wp-content/uploads/VisualizationTools.zip).

Deepa Krishnaswamy

Brigham and Women's Hospital

August 2025

Notes:
- Some of the bounding boxes have no lesion label. We keep them for now, and assign a label of Neoplasm.
- Some SOPInstanceUIDs from the annotations do not exist in IDC. Not sure where they are from. Known issue.  
- Some SOPInstanceUIDs have empty PixelSpacing and empty ImagePositionPatient. We keep these for now, even though they might not be useful later on.



In [ ]:
location = 'us-central1'
bucket_sr_directory = "gs://sr_lung_pet_ct_dx"

# Install highdicom from github

In [ ]:
!git clone https://github.com/imagingdatacommons/highdicom ~/highdicom

Cloning into '/root/highdicom'...
remote: Enumerating objects: 8294, done.
remote: Counting objects: 100% (1568/1568), done.
remote: Compressing objects: 100% (374/374), done.
remote: Total 8294 (delta 1330), reused 1255 (delta 1165), pack-reused 6726 (from 4)
Receiving objects: 100% (8294/8294), 8.14 MiB | 5.26 MiB/s, done.
Resolving deltas: 100% (5383/5383), done.


In [ ]:
!pip install ~/highdicom

Processing /root/highdicom
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.4 MB/s eta 0:00:00
  Created wheel for highdicom: filename=highdicom-0.26.1-py3-none-any.whl size=1106466 sha256=f1465db3eedb423b17d4180524df77bcd51ab26712947c75785d19294c8d5c63
  Stored in directory: /tmp/pip-ephem-wheel-cache-w0bbkgcl/wheels/e1/2a/ca/e605fe591961d04f0a82d0ca4d8593aaeeba598f4a9c806a09
Successfully built highdicom


# Parameterization

In [ ]:
#@title Enter your Project ID here
# initialize this variable with your Google Cloud Project ID!
project_name = "idc-external-018" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


# Environment setup

In [ ]:
import os

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# !pip install highdicom

In [ ]:
!pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 17.5 MB/s eta 0:00:00


In [ ]:
!pip install pydicom

In [ ]:
!pip install idc-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.3.2
    Uninstalling duckdb-1.3.2:
      Successfully uninstalled duckdb-1.3.2


In [ ]:
import os
import sys
import time

from google.cloud import bigquery
from google.cloud import storage

import numpy as np
import pandas as pd
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

import json
from pathlib import Path

import pydicom
from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

In [ ]:
from pydicom.dataset import Dataset

In [ ]:
from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion
)

from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation
)
# from highdicom.sr.value_types import (
#     CodedConcept,
#     CodeContentItem,
# )
# why from highdicom.sr.value_types for CodedConcept? Should be highdicom.sr.CodedConcept
from highdicom.sr import CodedConcept

from highdicom.sr import ImageLibrary
from highdicom.sr import ImageLibraryEntryDescriptors
from highdicom.sr.value_types import Code
from highdicom.sr.value_types import ContentItem
from highdicom.sr import PixelOriginInterpretationValues

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

In [ ]:
import highdicom as hr
hr.__version__

'0.26.1'

In [ ]:
pydicom.__version__

'3.0.1'

In [ ]:
import seaborn as sns

In [ ]:
from idc_index import IDCClient

idc_client = IDCClient.client()

# Read in the annotations from TCIA

In [ ]:
!wget "https://www.cancerimagingarchive.net/wp-content/uploads/Lung-PET-CT-Dx-Annotations-XML-Files-rev12222020.zip"

--2025-08-28 21:43:17--  https://www.cancerimagingarchive.net/wp-content/uploads/Lung-PET-CT-Dx-Annotations-XML-Files-rev12222020.zip
Resolving www.cancerimagingarchive.net (www.cancerimagingarchive.net)... 144.30.169.13
Connecting to www.cancerimagingarchive.net (www.cancerimagingarchive.net)|144.30.169.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17304054 (17M) [application/zip]
Saving to: ‘Lung-PET-CT-Dx-Annotations-XML-Files-rev12222020.zip’

Lung-PET-CT-Dx-Anno 100%[===================>]  16.50M  16.0MB/s    in 1.0s    

2025-08-28 21:43:18 (16.0 MB/s) - ‘Lung-PET-CT-Dx-Annotations-XML-Files-rev12222020.zip’ saved [17304054/17304054]



In [ ]:
%%capture
!unzip /content/Lung-PET-CT-Dx-Annotations-XML-Files-rev12222020.zip

# Process the annotations and join with IDC data

In [ ]:
import numpy as np
import os
from xml.etree import ElementTree
import re

def get_category(category_file):
    class_list = []
    with open(category_file, 'r') as f:
        for line in f.readlines():
            class_list.append(line.rstrip('\n'))

    return class_list


class XML_preprocessor(object):

    def __init__(self, data_path, num_classes, normalize=False):
        self.path_prefix = data_path
        self.num_classes = num_classes
        self.normalization = normalize
        self.data = dict()
        self._preprocess_XML()

    def _preprocess_XML(self):
        filenames = os.listdir(self.path_prefix)
        for filename in filenames:

            tree = ElementTree.parse(os.path.join(self.path_prefix, filename))
            root = tree.getroot()
            bounding_boxes = []
            class_names = []
            one_hot_classes = []
            size_tree = root.find('size')
            width = float(size_tree.find('width').text)
            height = float(size_tree.find('height').text)

            for object_tree in root.findall('object'):
              for bounding_box in object_tree.iter('bndbox'):
                if self.normalization:
                  xmin = float(bounding_box.find('xmin').text)/width
                  ymin = float(bounding_box.find('ymin').text)/height
                  xmax = float(bounding_box.find('xmax').text)/width
                  ymax = float(bounding_box.find('ymax').text)/height
                else:
                  xmin = float(bounding_box.find('xmin').text)
                  ymin = float(bounding_box.find('ymin').text)
                  xmax = float(bounding_box.find('xmax').text)
                  ymax = float(bounding_box.find('ymax').text)
              bounding_box = [xmin,ymin,xmax,ymax]
              bounding_boxes.append(bounding_box)

              class_name = object_tree.find('name').text
              if (class_name):
                class_names.append(class_name.upper())
              else:
                class_names.append("NA")

              one_hot_class = self._to_one_hot(class_name)
              # if no ones in the vector, that means unknown label, print out the filename with issues
              if (np.sum(one_hot_class)==0):
                print('Label not assigned in one hot vector for this filename: ' + str(image_name))
              one_hot_classes.append(one_hot_class)

            # image_name = root.find('filename').text
            image_name = filename
            # filename_tag = root.find('filename')
            # image_name = filename_tag.text if filename_tag is not None else filename
            bounding_boxes = np.asarray(bounding_boxes)
            one_hot_classes = np.asarray(one_hot_classes)
            image_data = np.hstack((bounding_boxes, one_hot_classes))
            self.data[image_name] = image_data

    def _to_one_hot(self, name):
        one_hot_vector = [0] * self.num_classes
        if name == 'A' or name == "a":
          one_hot_vector[0] = 1
        elif name == 'B' or name == "b":
          one_hot_vector[1] = 1
        elif name == 'E' or name == "e":
          one_hot_vector[2] = 1
        elif name == 'G' or name == "g":
          one_hot_vector[3] = 1
        else:
          print('unknown label: %s' %name)
          print()

        return one_hot_vector


## example on how to use it
# import pickle
# data = XML_preprocessor('VOC2007/Annotations/').data
# pickle.dump(data,open('VOC2007.p','wb'))

In [ ]:
# Parse the patient annotations and save in dictionary

xml_path = "/content/Annotation"
subjects = sorted(os.listdir(xml_path))

# For now remove these three subjects as they are causing issues with XML parsing
# subjects.remove("A0024")
# subjects.remove("A0192")
# subjects.remove("A0118")

num_subjects = len(subjects)
print('num_subjects: ' + str(num_subjects))

xml_data_patients = dict()
for subject in subjects:
  try:
    xml_data = XML_preprocessor(os.path.join(xml_path,subject), num_classes=4).data
  except:
    print('ERROR: Cannot parse annotations from subject ' + str(subject) + ', skipping subject')
  else:
    sop_uids = list(xml_data.keys())
    xml_data_patients[subject] = {}
    for sop_uid in sop_uids:
      label_sum = np.int8(np.sum(xml_data[sop_uid][0][4::]))
      # only add if label_sum = 1. if 0, then label was not assigned
      # if label_sum == 1:
      xml_data_patients[subject][sop_uid] = xml_data[sop_uid]


num_subjects: 358
ERROR: Cannot parse annotations from subject A0024, skipping subject
unknown label: Q

Label not assigned in one hot vector for this filename: 1.3.6.1.4.1.14519.5.2.1.6655.2359.213363836922656849158065964900.xml
unknown label: Q

Label not assigned in one hot vector for this filename: 1.3.6.1.4.1.14519.5.2.1.6655.2359.101178658572428842030477561194.xml
unknown label: Q

Label not assigned in one hot vector for this filename: 1.3.6.1.4.1.14519.5.2.1.6655.2359.765854041543259519493495003088.xml
unknown label: Q

Label not assigned in one hot vector for this filename: 1.3.6.1.4.1.14519.5.2.1.6655.2359.115968120059620226755041374609.xml
unknown label: Q

Label not assigned in one hot vector for this filename: 1.3.6.1.4.1.14519.5.2.1.6655.2359.516234543888758394406094956564.xml
unknown label: Q

Label not assigned in one hot vector for this filename: 1.3.6.1.4.1.14519.5.2.1.6655.2359.186177003150212236907165415466.xml
unknown label: Q

Label not assigned in one hot vector 

In [ ]:
# Initialize the annot_df dataframe

annot_df = pd.DataFrame()
PatientID_list = []
SOPInstanceUID_list = []
xml_filename_list = []
xmin_list = []
xmax_list = []
ymin_list = []
ymax_list = []
class_names = []

# Form the lists needed for the dataframe
for PatientID in list(xml_data_patients.keys()):
  # get dict of patient
  xml_data_patient = xml_data_patients[PatientID]
  # SOPInstanceUID.xml
  for filename in list(xml_data_patient.keys()):
    # get dict of sops
    xml_data_sop = xml_data_patient[filename]
    xml_filename = os.path.join(xml_path, PatientID, filename)
    one_hot_vector = xml_data_sop[0][4::]
    if one_hot_vector[0] == 1:
      class_name = "A"
    elif one_hot_vector[1] == 1:
      class_name = "B"
    elif one_hot_vector[2] == 1:
      class_name = "E"
    elif one_hot_vector[3] == 1:
      class_name = "G"
    else:
      class_name = "NA"
    # append all
    PatientID_list.append(PatientID)
    SOPInstanceUID_list.append(os.path.splitext(filename)[0])
    xml_filename_list.append(xml_filename)
    xmin_list.append(xml_data_sop[0][0])
    ymin_list.append(xml_data_sop[0][1])
    xmax_list.append(xml_data_sop[0][2])
    ymax_list.append(xml_data_sop[0][3])
    # xmax_list.append(xml_data_sop[0][1])
    # ymin_list.append(xml_data_sop[0][2])
    class_names.append(class_name)

# Fill in the dataframe
annot_df['PatientID'] = PatientID_list
annot_df['SOPInstanceUID'] = SOPInstanceUID_list
annot_df['xml_filename'] = xml_filename_list
annot_df['xmin'] = xmin_list
annot_df['xmax'] = xmax_list
annot_df['ymin'] = ymin_list
annot_df['ymax'] = ymax_list
annot_df['class_name'] = class_names

print(len(annot_df))

31444


In [ ]:
# Get the following fields using bigQuery:
#   StudyInstanceUID
#   SeriesInstanceUID
#   IPP[0] and IPP[1]
#   Pixel spacing
#   Dim

client_bq = bigquery.Client(project=project_name)

query = f"""
    SELECT
      PatientID,
      StudyInstanceUID,
      SeriesInstanceUID,
      SeriesDescription,
      SOPInstanceUID,
      ImageType
    FROM
      `bigquery-public-data.idc_current.dicom_all`
    WHERE
      SOPInstanceUID IN UNNEST(@SOPInstanceUID_list)
    ORDER BY
      PatientID,
      StudyInstanceUID,
      SeriesInstanceUID,
      ImagePositionPatient[SAFE_OFFSET(2)]
      """

job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("SOPInstanceUID_list", "STRING", SOPInstanceUID_list)])
annot_df_idc = client_bq.query(query, job_config=job_config).to_dataframe()
print(len(annot_df_idc))


30630


In [ ]:
# Get the SOPInstanceUIDs that are in annot_df and not in annot_df_idc
# Known problem with some of the data!

SOPInstanceUID_list_missing = list(set(annot_df['SOPInstanceUID'].values) - set(annot_df_idc['SOPInstanceUID'].values))
print(len(SOPInstanceUID_list_missing))
print(SOPInstanceUID_list_missing[0])

652
1.3.6.1.4.1.14519.5.2.1.6655.2359.235871856958807701793743345140


In [ ]:
# # Since we cannot create bounding boxes with empty PixelSpacing and empty ImagePositionPatient,
# # we remove these SOPInstanceUIDs from the dataframe

# annot_df_idc = annot_df_idc[annot_df_idc['PixelSpacing'].map(len) > 0]
# annot_df_idc = annot_df_idc[annot_df_idc['ImagePositionPatient'].map(len) > 0]

In [ ]:
# # Create separate columns for pixel spacing and image position patient
# # pixel_spacing_x, pixel_spacing_y, ipp0, ipp1, ipp2

# annot_df_idc['pixel_spacing_x'] = [np.float32(f[0]) for f in annot_df_idc['PixelSpacing'].values]
# annot_df_idc['pixel_spacing_y'] = [np.float32(f[1]) for f in annot_df_idc['PixelSpacing'].values]
# annot_df_idc['ipp0'] = [np.float32(f[0]) for f in annot_df_idc['ImagePositionPatient'].values]
# annot_df_idc['ipp1'] = [np.float32(f[1]) for f in annot_df_idc['ImagePositionPatient'].values]
# annot_df_idc['ipp2'] = [np.float32(f[2]) for f in annot_df_idc['ImagePositionPatient'].values]

# annot_df_idc = annot_df_idc[["PatientID", "StudyInstanceUID", "SeriesInstanceUID", "SOPInstanceUID",
#                              "num_rows", "num_columns",
#                              "pixel_spacing_x", "pixel_spacing_y",
#                              "ipp0", "ipp1", "ipp2"]]
# print(len(annot_df_idc))

In [ ]:
# Now we create a dataframe with the lung location
# We put lung for everything, with no topographical modifier

# finding_site_lung = ["39607008", "SCT", "Lung structure (body structure)"]
# finding_site_lung = codes.SCT.LungStructure
# Need to shorten
finding_site_lung = ["39607008", "SCT", "Lung structure"]
finding_site_lung_top_mod = []

annot_df_idc['FindingSite'] = [finding_site_lung] * len(annot_df_idc)
annot_df_idc['FindingSite_topographicalModifier'] = [""] * len(annot_df_idc)

# Add in the TrackingIdentifier
# For now we set every SOP to a different tracking identifier - later could change to be per lesion.
tracking_identifier = []
SeriesInstanceUID_list = annot_df_idc.drop_duplicates(subset='SeriesInstanceUID')['SeriesInstanceUID'].values
for n in range(0,len(SeriesInstanceUID_list)):
  SOPInstanceUID_list = annot_df_idc[annot_df_idc['SeriesInstanceUID']==SeriesInstanceUID_list[n]]['SOPInstanceUID'].values # these will be unique
  num_sop = len(SOPInstanceUID_list)
  tracking_identifier.append(list(range(1,num_sop+1,1)))
tracking_identifier_list = [item for sublist in tracking_identifier for item in sublist]
annot_df_idc['TrackingIdentifier'] = tracking_identifier_list
print(len(annot_df_idc))

30630


In [ ]:
print(len(list(set(annot_df_idc['SOPInstanceUID'].values))))

30630


In [ ]:
# Now we join the annot_df_idc_keep with the annot_df

df_lung = pd.merge(left=annot_df_idc, right=annot_df, how='left', on="SOPInstanceUID")
print(len(df_lung))
print(len(list(set(df_lung['SOPInstanceUID'].values))))

30776
30630


In [ ]:
# Now we add in the type of lesion - based on the class_names

finding_types = []

for n in range(0,len(df_lung)):
  class_name = df_lung['class_name'].values[n]
  if class_name == 'A':
    finding_type = codes.SCT.Adenocarcinoma
  elif class_name == 'B':
    finding_type = codes.SCT.SmallCellCarcinoma
  elif class_name == 'E':
    finding_type = codes.SCT.LargeCellCarcinoma
  elif class_name == 'G':
    finding_type = codes.SCT.SquamousCellCarcinoma
  else:
    finding_type = codes.SCT.Neoplasm
  finding_types.append(finding_type)

df_lung['FindingType'] = finding_types

In [ ]:
# keep specific columns in order

df_lung = df_lung[["PatientID_x", "StudyInstanceUID", "SeriesInstanceUID", "SOPInstanceUID",
                   "xmin", "xmax", "ymin", "ymax",
                   "xml_filename",
                   "TrackingIdentifier",
                   "class_name",
                   'FindingSite',
                   'FindingSite_topographicalModifier',
                   'FindingType']]
df_lung = df_lung.rename(columns={'PatientID_x': 'PatientID'})

In [ ]:
df_lung.head()

,PatientID,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,xmin,xmax,ymin,ymax,xml_filename,TrackingIdentifier,class_name,FindingSite,FindingSite_topographicalModifier,FindingType
0,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021...,1.3.6.1.4.1.14519.5.2.1.6655.2359.741206086241...,296.0,351.0,331.0,382.0,/content/Annotation/A0001/1.3.6.1.4.1.14519.5....,1,A,"[39607008, SCT, Lung structure]",,"(1187332001, SCT, Adenocarcinoma, None)"
1,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021...,1.3.6.1.4.1.14519.5.2.1.6655.2359.289186849349...,293.0,348.0,318.0,384.0,/content/Annotation/A0001/1.3.6.1.4.1.14519.5....,2,A,"[39607008, SCT, Lung structure]",,"(1187332001, SCT, Adenocarcinoma, None)"
2,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021...,1.3.6.1.4.1.14519.5.2.1.6655.2359.191468104323...,285.0,355.0,319.0,389.0,/content/Annotation/A0001/1.3.6.1.4.1.14519.5....,3,A,"[39607008, SCT, Lung structure]",,"(1187332001, SCT, Adenocarcinoma, None)"
3,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021...,1.3.6.1.4.1.14519.5.2.1.6655.2359.253763111879...,289.0,353.0,317.0,391.0,/content/Annotation/A0001/1.3.6.1.4.1.14519.5....,4,A,"[39607008, SCT, Lung structure]",,"(1187332001, SCT, Adenocarcinoma, None)"
4,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021...,1.3.6.1.4.1.14519.5.2.1.6655.2359.102500633407...,286.0,355.0,310.0,402.0,/content/Annotation/A0001/1.3.6.1.4.1.14519.5....,5,A,"[39607008, SCT, Lung structure]",,"(1187332001, SCT, Adenocarcinoma, None)"


# Functions

In [ ]:
##################################
### calculate_bbox_coordinates ###
##################################

# def calculate_bbox_coordinates(xmin,
#                                xmax,
#                                ymin,
#                                ymax,
#                                ipp_0,
#                                ipp_1,
#                                pixel_spacing_x,
#                                pixel_spacing_y):

#   """
#   Summary:
#     Calculates the bbox and the center needed for the SRs
#   Inputs:
#     xmin            - xmin coordinate
#     xmax            - xmax coordinate
#     ymin            - ymin coordinate
#     ymax            - ymax coordinate
#     ipp_0           - ImagePositionPatient[0]
#     ipp_1           - ImagePositionPatient[1]
#     pixel_spacing_x - pixel spacing in x direction
#     pixel_spacing_y - pixel spacing in y direction
#   Outputs:
#     bbox            - bounding box of 4 2D points - array of 4 pairs of points
#     bbox_center     - single 2D point of the center of the bounding box
#   """

#   # lower left
#   bbox0 = (xmin * pixel_spacing_x) + ipp_0
#   bbox1 = (ymin * pixel_spacing_y) + ipp_1
#   # lower right
#   bbox2 = (xmax * pixel_spacing_x) + ipp_0
#   bbox3 = (ymin * pixel_spacing_y) + ipp_1
#   # upper right
#   bbox4 = (xmax * pixel_spacing_x) + ipp_0
#   bbox5 = (ymax * pixel_spacing_y) + ipp_1
#   # upper left
#   bbox6 = (xmin * pixel_spacing_x) + ipp_0
#   bbox7 = (ymax * pixel_spacing_y) + ipp_1
#   # Define bbox coordinates
#   # lower left, lower right, upper right, upper left
#   bbox = np.array([[bbox0, bbox1], [bbox2, bbox3], [bbox4, bbox5], [bbox6, bbox7]])

#   # Define bbox center
#   bbox_center_x = (((xmax-xmin)/2) * pixel_spacing_x) + ipp_0
#   bbox_center_y = (((ymax-ymin)/2) * pixel_spacing_y) + ipp_0
#   bbox_center = np.array([bbox_center_x, bbox_center_y])

#   return bbox, bbox_center

def calculate_bbox_coordinates(xmin,
                               xmax,
                               ymin,
                               ymax):
  """
  Summary:
    Calculates the bbox and the center needed for the SRs
  Inputs:
    xmin            - xmin coordinate
    xmax            - xmax coordinate
    ymin            - ymin coordinate
    ymax            - ymax coordinate
  Outputs:
    bbox            - bounding box of 4 2D points - array of 4 pairs of points
    bbox_center     - single 2D point of the center of the bounding box
  """

  # point 1 (x,y) = lower left
  # point 2 (x,y) = lower right
  # point 3 (x,y) = upper right
  # point 4 (x,y) = upper left

  # lower left
  bbox0 = xmin
  bbox1 = ymin
  # lower right
  bbox2 = xmax
  bbox3 = ymin
  # upper right
  bbox4 = xmax
  bbox5 = ymax
  # upper left
  bbox6 = xmin
  bbox7 = ymax
  # Define bbox coordinates
  # lower left, lower right, upper right, upper left
  # and repeat the first point
  bbox = np.array([[bbox0, bbox1], [bbox2, bbox3], [bbox4, bbox5], [bbox6, bbox7], [bbox0, bbox1]])

  # Define bbox center
  bbox_center_x = (xmax-xmin)/2 + xmin
  bbox_center_y = (ymax-ymin)/2 + ymin
  bbox_center = np.array([bbox_center_x, bbox_center_y])

  return bbox, bbox_center

#############################
### save_SR_for_case_bbox ###
#############################

def save_SR_for_case_bbox(sop_files,
                          df_lung_subset,
                          path_sr):
  """
  Summary:
  This function creates and saves a bbox SR.

  Inputs:
    sop_files            - all of the CT filenames
    df_lung_subset       - the dataframe containing the uids, and info needed for the
                         bounding boxes
    path_sr              - filename for the output SR

  """


  # Read CT Image data set from PS3.10 files on disk
  # image_dataset = dcmread(sop_filename_keep)

  # image_dataset = dcmread(sop_filenames_keep[0]) # can be any?
  # provide all frames for image_dataset instead of just one
  image_dataset = [pydicom.dcmread(f, stop_before_pixels=True) for f in sop_files]

  # Describe the context of reported observations: the person that reported
  # the observations and the device that was used to make the observations
  observer_person_context = ObserverContext(
      observer_type=codes.DCM.Person,
      observer_identifying_attributes=PersonObserverIdentifyingAttributes(
          name='Anonymous^Reader'
      )
  )
  observer_device_context = ObserverContext(
      observer_type=codes.DCM.Device,
      observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
          uid=generate_uid()
      )
  )
  observation_context = ObservationContext(
      observer_person_context=observer_person_context,
      #observer_device_context=observer_device_context,
  )

  # # Add in the clinical information
  # ClinicalTrialProtocolID = "10.7937/TCIA.2020.NNC2-0461"
  # IssuerOfClinicalTrialProtocolID = "DOI"
  # ClinicalTrialSponsorName = "National Cancer Institute"

  # Add in the clinical information
  ClinicalTrialProtocolID = "10.5281/zenodo.16989820"
  IssuerOfClinicalTrialProtocolID = "DOI"
  ClinicalTrialSponsorName = "Imaging Data Commons"

  imaging_measurements = []

  SeriesInstanceUID = df_lung_subset['SeriesInstanceUID'].values[0]

  ########################
  ### Add for each SOP ###
  ########################

  # SOPInstanceUID_list = list(set(df_lung_subset['SOPInstanceUID'].values))
  # Maintain the same order
  df_lung_subset_temp = df_lung_subset.drop_duplicates(subset='SOPInstanceUID')
  SOPInstanceUID_list = list(df_lung_subset_temp['SOPInstanceUID'].values)
  num_sop = len(SOPInstanceUID_list)

  for n in range(0,num_sop):

    SOPInstanceUID = SOPInstanceUID_list[n]
    df_lung_subset_sop = df_lung_subset[df_lung_subset['SOPInstanceUID']==SOPInstanceUID]

    #################################
    ### Calculate the coordinates ###
    #################################

    # Get the SOPInstanceUID and filename
    image_filename = df_lung_subset_sop['filename'].values[0]
    im = pydicom.dcmread(image_filename, stop_before_pixels=True)
    SeriesNumber = im.SeriesNumber

    #################################
    ### Now for each bbox per sop ###
    #################################

    num_bbox = len(df_lung_subset_sop)

    for m in range(0,num_bbox):

      xmin = df_lung_subset_sop['xmin'].values[m]
      xmax = df_lung_subset_sop['xmax'].values[m]
      ymin = df_lung_subset_sop['ymin'].values[m]
      ymax = df_lung_subset_sop['ymax'].values[m]


      finding_site = df_lung_subset_sop['FindingSite'].values[m]
      finding_site_top_mod = df_lung_subset_sop['FindingSite_topographicalModifier'].values[m]
      finding_type = df_lung_subset['FindingType'].values[m]

      identifier_index = df_lung_subset_sop['TrackingIdentifier'].values[m]

      bbox,_ = calculate_bbox_coordinates(xmin,
                                          xmax,
                                          ymin,
                                          ymax)

      geometric_purpose = CodedConcept(value="75958009",
                                       scheme_designator="SCT",
                                       meaning="Bounded by")

      referenced_region_bbox = ImageRegion(graphic_type=GraphicTypeValues.POLYLINE,
                                           graphic_data=bbox,
                                           source_image=SourceImageForRegion.from_source_image(image=im,
                                                                                               referenced_frame_numbers=None),
                                           pixel_origin_interpretation=None)

      finding_location = CodedConcept(value=finding_site[0],
                                      scheme_designator=finding_site[1],
                                      meaning=finding_site[2])
      # if no topographical modifier
      if (finding_site_top_mod==""):
        finding_sites = [
            FindingSite(anatomic_location=finding_location)
        ]
      # if topographical modifier
      else:
        topographical_modifier = CodedConcept(value=finding_site_top_mod[0],
                                              scheme_designator=finding_site_top_mod[1],
                                              meaning=finding_site_top_mod[2])
        topographical_modifier = finding_site_top_mod
        finding_sites = [
            FindingSite(anatomic_location=finding_location,
                        topographical_modifier=topographical_modifier)
        ]

      imaging_measurements.append(
          PlanarROIMeasurementsAndQualitativeEvaluations(
              tracking_identifier=TrackingIdentifier(
                  uid=generate_uid(),
                  identifier=str(identifier_index)
              ),
              referenced_region=referenced_region_bbox,
              geometric_purpose=geometric_purpose,
              finding_type=finding_type,
              finding_sites=finding_sites
          )
      )



  # print(len(imaging_measurements))

  # If no SOPInstanceUIDs had bounding boxes, return.
  if (len(imaging_measurements)==0):
    return

  else:

    # Create the report content
    procedure_code = CodedConcept(value="363679005",
                                  scheme_designator="SCT",
                                  meaning="Imaging procedure")
    measurement_report = MeasurementReport(
        observation_context=observation_context,
        procedure_reported=procedure_code,
        imaging_measurements=imaging_measurements
    )

    # Create the Structured Report instance
    series_instance_uid = generate_uid()
    sr_dataset = Comprehensive3DSR(
        # evidence=[image_dataset],
        evidence=image_dataset,
        # content=measurement_report[0],
        content=measurement_report,
        series_number=SeriesNumber+100,
        series_instance_uid=series_instance_uid,
        sop_instance_uid=generate_uid(),
        # series_description="Bounding box annotation of the lesion",
        series_description="Tumor bounding box",
        instance_number=1,
        manufacturer='Expert annotation from TCIA',
        is_complete=True,
        is_final=True
    )

    # Add clinical dataset information
    sr_dataset.ClinicalTrialProtocolID=ClinicalTrialProtocolID,
    sr_dataset.IssuerOfClinicalTrialProtocolID=IssuerOfClinicalTrialProtocolID,
    sr_dataset.ClinicalTrialSponsorName=ClinicalTrialSponsorName
    # sr_dataset.OtherClinicalTrialProtocolIDsSequence = []
    # # function to add IDs
    # def add_other_protocol(ds, protocol_id: str, issuer: str):
    #   item = Dataset()
    #   item.ClinicalTrialProtocolID = protocol_id
    #   item.IssuerOfClinicalTrialProtocolID = issuer
    #   ds.OtherClinicalTrialProtocolIDsSequence.append(item)
    # # Add the additional IDs
    # add_other_protocol(sr_dataset, ClinicalTrialProtocolID, IssuerOfClinicalTrialProtocolID)

    sr_dataset.save_as(path_sr, enforce_file_format=True)

    return



# Create bounding box SRs for Lung-PET-CT-Dx

In [ ]:
# Get the list of SeriesInstanceUIDs to create SRs for

SeriesInstanceUIDs = list(set(df_lung['SeriesInstanceUID'].values))
num_SeriesInstanceUIDs = len(SeriesInstanceUIDs)
print('num_SeriesInstanceUIDs: ' + str(num_SeriesInstanceUIDs))

PatientIDs = list(set(df_lung['PatientID'].values))
num_patients = len(PatientIDs)
print('num_patients: ' + str(num_patients))

num_SeriesInstanceUIDs: 1087
num_patients: 353


In [ ]:
PatientID = "Lung_Dx-A0156"
SeriesInstanceUIDs_temp = list(set(df_lung[df_lung['PatientID']==PatientID]['SeriesInstanceUID'].values))

print('PatientID: ' + str(PatientID))
print('SeriesInstanceUIDs_temp: ' + str(SeriesInstanceUIDs_temp))

PatientID: Lung_Dx-A0156
SeriesInstanceUIDs_temp: ['1.3.6.1.4.1.14519.5.2.1.6655.2359.317266616336785850519921402078', '1.3.6.1.4.1.14519.5.2.1.6655.2359.834859555452290849106199077407']


In [ ]:
# Create the SRs

sr_directory = "/content/sr"
if not os.path.isdir(sr_directory):
  os.mkdir(sr_directory)

checkpoints = {int(len(SeriesInstanceUIDs) * i / 10) for i in range(1, 11)}

for index,SeriesInstanceUID in enumerate(SeriesInstanceUIDs,1):

  print('*** On series ' + str(index) + ': ' + str(SeriesInstanceUID) + ' ***')

  # The subset of the dataframe for the SeriesInstanceUID
  df_lung_subset = df_lung[df_lung['SeriesInstanceUID']==SeriesInstanceUID]

  #######################
  ### Output filename ###
  #######################

  # Corresponding PatientID
  PatientID = df_lung_subset['PatientID'].values[0]
  print('PatientID: ' + str(PatientID))

  # List of SOPInstanceUIDs for the particular series
  SOPInstanceUID_list_boxes = df_lung_subset['SOPInstanceUID'].values

  # Path of structured report
  path_sr = os.path.join(sr_directory, PatientID, SeriesInstanceUID + '.dcm')
  if not os.path.isdir(os.path.join(sr_directory, PatientID)):
    os.makedirs(os.path.join(sr_directory, PatientID), exist_ok=True)
  path_sr_bucket = os.path.join(bucket_sr_directory, PatientID, SeriesInstanceUID + '.dcm')

  #####################################
  ### Download and process CT files ###
  #####################################

  # Create dicom directory
  dicom_directory = "/content/ct"
  if not os.path.isdir(dicom_directory):
    os.mkdir(dicom_directory)

  # Download series
  # Can't download a single sop.
  idc_client.download_from_selection(
      seriesInstanceUID=SeriesInstanceUID,
      downloadDir=dicom_directory,
      dirTemplate='',
      quiet=True
  )

  # Get the list of files
  sop_files = os.listdir("/content/ct")
  sop_files = [os.path.join("/content/ct",f) for f in sop_files]

  # read all the sop_files
  # get the filename, and the ipp
  SOPInstanceUID_list_all = []
  IPP_list_all = []
  ipp2 = []
  for sop_file in sop_files:
    sop_data = pydicom.dcmread(sop_file)
    SOPInstanceUID_list_all.append(sop_data.SOPInstanceUID)
    try:
      IPP_list_all.append(sop_data.ImagePositionPatient)
      ipp2.append(sop_data.ImagePositionPatient[2])
    except:
      IPP_list_all.append(0)
      ipp2.append(0)

  # Now find the filenames and IPP of the SOPInstanceUIDs that match
  sop_indices = [SOPInstanceUID_list_all.index(f) for f in SOPInstanceUID_list_boxes]
  sop_files_keep = [sop_files[i] for i in sop_indices]
  ipp2_keep = [ipp2[i] for i in sop_indices]
  # ipp = [IPP_list_all[i] for i in sop_indices]
  # ipp0 = [f[0] for f in ipp]
  # ipp1 = [f[1] for f in ipp]
  # ipp2 = [f[2] for f in ipp]

  # Now add to dataframe
  df_lung_subset['filename'] = sop_files_keep
  df_lung_subset['ipp2'] = ipp2_keep

  # Let's override the TrackingIdentifier - we order by the ipp2
  df_lung_subset = df_lung_subset.sort_values(by=['ipp2'])
  df_lung_subset['TrackingIdentifier'] = list(range(1,len(df_lung_subset)+1,1))

  #####################
  ### Create the SR ###
  #####################

  save_SR_for_case_bbox(sop_files=sop_files,
                        df_lung_subset=df_lung_subset,
                        path_sr=path_sr)

  # copy the SR to the bucket
  if os.path.exists(path_sr):
    try:
      !gsutil cp $path_sr $path_sr_bucket
    except:
      print('ERROR: cannot copy from ' + str(path_sr) + ' to: ' + str(path_sr_bucket))

  # delete the dicom directory after processing
  !rm -r $dicom_directory

  if index in checkpoints:
    print(f"{(index / len(SeriesInstanceUIDs)) * 100:.0f}% of series processed.")


*** On series 1: 1.3.6.1.4.1.14519.5.2.1.6655.2359.254261552466081472301853188947 ***
PatientID: Lung_Dx-A0262


/tmp/ipython-input-3027297422.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['filename'] = sop_files_keep
/tmp/ipython-input-3027297422.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['ipp2'] = ipp2_keep
/usr/local/lib/python3.12/dist-packages/highdicom/base.py:165: UserWarning: The string "Lung_Dx-A0262" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https:/

Copying file:///content/sr/Lung_Dx-A0262/1.3.6.1.4.1.14519.5.2.1.6655.2359.254261552466081472301853188947.dcm [Content-Type=application/dicom]...
/ [1 files][ 65.2 KiB/ 65.2 KiB]                                                
Operation completed over 1 objects/65.2 KiB.                                     
*** On series 2: 1.3.6.1.4.1.14519.5.2.1.6655.2359.562493571362168108950325066949 ***
PatientID: Lung_Dx-A0185


/tmp/ipython-input-3027297422.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['filename'] = sop_files_keep
/tmp/ipython-input-3027297422.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['ipp2'] = ipp2_keep
/usr/local/lib/python3.12/dist-packages/highdicom/base.py:165: UserWarning: The string "Lung_Dx-A0185" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https:/

Copying file:///content/sr/Lung_Dx-A0185/1.3.6.1.4.1.14519.5.2.1.6655.2359.562493571362168108950325066949.dcm [Content-Type=application/dicom]...
/ [1 files][ 59.8 KiB/ 59.8 KiB]                                                
Operation completed over 1 objects/59.8 KiB.                                     
*** On series 3: 1.3.6.1.4.1.14519.5.2.1.6655.2359.112642828995873127870881326817 ***
PatientID: Lung_Dx-G0042


...

Copying file:///content/sr/Lung_Dx-G0032/1.3.6.1.4.1.14519.5.2.1.6655.2359.215777061776507799548805220588.dcm [Content-Type=application/dicom]...
/ [1 files][ 20.1 KiB/ 20.1 KiB]                                                
Operation completed over 1 objects/20.1 KiB.                                     
*** On series 802: 1.3.6.1.4.1.14519.5.2.1.6655.2359.603182895431999479265912409492 ***
PatientID: Lung_Dx-A0131


/tmp/ipython-input-3027297422.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['filename'] = sop_files_keep
/tmp/ipython-input-3027297422.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['ipp2'] = ipp2_keep
/usr/local/lib/python3.12/dist-packages/highdicom/base.py:165: UserWarning: The string "Lung_Dx-A0131" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https:/

Copying file:///content/sr/Lung_Dx-A0131/1.3.6.1.4.1.14519.5.2.1.6655.2359.603182895431999479265912409492.dcm [Content-Type=application/dicom]...
/ [1 files][ 74.1 KiB/ 74.1 KiB]                                                
Operation completed over 1 objects/74.1 KiB.                                     
*** On series 803: 1.3.6.1.4.1.14519.5.2.1.6655.2359.287498281871506188442456699531 ***
PatientID: Lung_Dx-A0050


/tmp/ipython-input-3027297422.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['filename'] = sop_files_keep
/tmp/ipython-input-3027297422.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lung_subset['ipp2'] = ipp2_keep
/usr/local/lib/python3.12/dist-packages/highdicom/base.py:165: UserWarning: The string "Lung_Dx-A0050" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https:/

Copying file:///content/sr/Lung_Dx-A0050/1.3.6.1.4.1.14519.5.2.1.6655.2359.287498281871506188442456699531.dcm [Content-Type=application/dicom]...
/ [1 files][ 12.6 KiB/ 12.6 KiB]                                                
Operation completed over 1 objects/12.6 KiB.                                     
*** On series 804: 1.3.6.1.4.1.14519.5.2.1.6655.2359.196600760498278440810850906562 ***
PatientID: Lung_Dx-A0113
